## 1. IMPORTANDO LIBRERIAS

In [2]:
#libraries
from random import randint
import numpy as np
import re
import copy

## 2. EXTRACCIÓN DE LA DATA DEL ARCHIVO TXT

In [3]:
#2.1 Extraemos los nombres de las ciudades y 
#    realizamos el encoding con ayuda de un diccionario
encode_routes_dict={}
cnt=0
with open('rutas.txt', 'r') as fp:
    for line in fp:
        text_in_line=(re.split(r'\s',line))
        x=text_in_line[0]
        y=text_in_line[2]
        if x in encode_routes_dict:
            continue
        else: 
            encode_routes_dict[x]=cnt
            cnt=cnt+1
        if y in encode_routes_dict:
            continue
        else: 
            encode_routes_dict[y]=cnt
            cnt=cnt+1
#Creacion del diccionario invertido que se usará luego
inv_encode_routes_dict = {v: k for k, v in encode_routes_dict.items()}


In [4]:
print("Dictionario de ciudades",encode_routes_dict)
print("Diccionario invertido de ciudades",inv_encode_routes_dict)

Dictionario de ciudades {'AlphaCentauri': 0, 'Snowdin': 1, 'Tambi': 2, 'Faerun': 3, 'Norrath': 4, 'Straylight': 5, 'Tristram': 6, 'Arbre': 7}
Diccionario invertido de ciudades {0: 'AlphaCentauri', 1: 'Snowdin', 2: 'Tambi', 3: 'Faerun', 4: 'Norrath', 5: 'Straylight', 6: 'Tristram', 7: 'Arbre'}


## 3. Creacion de la matrix de conexiones

In [5]:
#3.1 Funcion para encontrar el encoding respectivo de cada ciudad
def encode_city(city):
    return encode_routes_dict[city]
con=np.zeros((cnt+1,cnt+1))
with open('rutas.txt', 'r') as fp:
    for line in fp:
        text_in_line=(re.split(r'\s',line))
        c1=encode_city(text_in_line[0])
        c2=encode_city(text_in_line[2])
        c3=int(text_in_line[4])
        con[c1,c2]=c3
        con[c2,c1]=c3
for i in range(0,con.shape[0]):
    con[i][i]=-1
con_list=list()
for i in range(con.shape[0]):
    con_list.append(list(con[i,:]))

In [6]:
con_list

[[-1.0, 66.0, 28.0, 60.0, 34.0, 34.0, 3.0, 108.0, 0.0],
 [66.0, -1.0, 22.0, 12.0, 91.0, 121.0, 111.0, 71.0, 0.0],
 [28.0, 22.0, -1.0, 39.0, 113.0, 130.0, 35.0, 40.0, 0.0],
 [60.0, 12.0, 39.0, -1.0, 63.0, 21.0, 57.0, 83.0, 0.0],
 [34.0, 91.0, 113.0, 63.0, -1.0, 9.0, 50.0, 60.0, 0.0],
 [34.0, 121.0, 130.0, 21.0, 9.0, -1.0, 27.0, 81.0, 0.0],
 [3.0, 111.0, 35.0, 57.0, 50.0, 27.0, -1.0, 90.0, 0.0],
 [108.0, 71.0, 40.0, 83.0, 60.0, 81.0, 90.0, -1.0, 0.0],
 [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -1.0]]

## 4. Solución del problema utilizando programacion dinámica

#### La estrategia utilizada para este problema es resolver el problema TSP(traveling sales problem)
#### pero se le agregar un nodo mas (inservible) el cual poseerá distancias "0" con los otros nodos y que 
#### posteriormente sera removido para encontrar la ruta optima

In [7]:
#4.1 Funciones necesarias para la obtencion de la rupta optima

#Funcion para encontrar en nodo "inservible"
def finding_break(lista):
    for i in range(len(lista)):
        if lista[i]==len(lista)-1:
            j=i
    return j        
#Funcion para la creacion de tuplas solucion
def tuples_solution(lista,j):
    nn=list()
    for i in range(0,len(lista)-1):
        if i<j-1:
            nn.append([lista[i],lista[i+1]])
        if i==j-1:
            continue
        if i>j:
            nn.append([lista[i],lista[i+1]])
    return nn

#Funcion para encontrar el camino optimo 
def finding_path(lista,inicio,final):
    tt=list()
    y=inicio
    for j in range(len(lista)):
        for i in range(len(lista)):
            if (lista[i][0])==y:
                tt.append(lista[i][0])
                y=lista[i][1]
    tt.append(final)
    return tt
#Funcion para poder visualizar el camino optimo con os nombres de las ciudades
def inv_encoding_path(ss):
    pp=list()
    for i in range(len(ss)):
        pp.append(inv_encode_routes_dict[ss[i]-1])
    return pp
#Funcion para obtener la distancia optima
def optimal_path_distance(lista_pares):
    suma=0
    for i in range(len(lista_pares)):
        actual_list=lista_pares[i]
        c_x=actual_list[0]-1
        c_y=actual_list[1]-1
        suma=con_list[c_x][c_y] + suma
    return suma
    

In [8]:
## RESOLUCION DEL PROBLEMA UTILIZANDO PROGRAMACION DINAMICA
matrix = con_list    
data = list(range(1,len(matrix)+1))
n = len(data)
all_sets = []
g = {}
p = []
#variable global que almacenara en primera instancia el camino solucion utilizando el nodo "inservible"
global ps
ps=list()
def main():
    for x in range(1, n):
        g[x + 1, ()] = matrix[x][0]
    #Seteo manual
    obtener_minimo(1,(2,3,4,5,6,7,8,9))
    solution = p.pop()
    ps.append(1)
    ps.append(solution[1][0])
    for x in range(n - 2):
        for new_solution in p:
            if tuple(solution[1]) == new_solution[0]:
                solution = new_solution
                ps.append(solution[1][0])
                break
    ps.append(1)
    return


def obtener_minimo(k, a):
    if (k, a) in g:
        # Already calculated Set g[%d, (%s)]=%d' % (k, str(a), g[k, a]))
        return g[k, a]

    values = []
    all_min = []
    for j in a:
        set_a = copy.deepcopy(list(a))
        set_a.remove(j)
        all_min.append([j, tuple(set_a)])
        result = obtener_minimo(j, tuple(set_a))
        values.append(matrix[k-1][j-1] + result)

    # get minimun value from set as optimal solution for
    g[k, a] = min(values)
    p.append(((k, a), all_min[values.index(g[k, a])]))

    return g[k, a]
main() 

In [9]:
#ULTIMOS PASOS PARA LA VISUALIZACION DEL PROBLEMA

In [10]:
ts=tuples_solution(ps,finding_break(ps))
inicio=ps[finding_break(ps)+1]
final=ps[finding_break(ps)-1]

In [11]:
path_not_encoding=finding_path(ts,inicio,final)

In [12]:
print("Ruta Optima: ")
print("->".join(inv_encoding_path(path_not_encoding)))
print("Valor optimo: ",optimal_path_distance(ts))

Ruta Optima: 
Tristram->AlphaCentauri->Norrath->Straylight->Faerun->Snowdin->Tambi->Arbre
Valor optimo:  141.0
